In [ ]:
%%bash
pip install -q pandas scikit-learn numpy matplotlib seaborn


### Baseline: TF-IDF + Logistic Regression
This notebook loads the teacher/course sentiment data, builds aspect-aware TF-IDF features, and trains a linear classifier.


In [ ]:
%%bash
set -e
python ../../sentiment_experiment.py --data ../../data/teacher_course.csv --output ../../outputs/baseline/joint --mode joint


In [ ]:
%%bash
set -e
python ../../sentiment_experiment.py --data ../../data/teacher_course.csv --output ../../outputs/baseline/aspect --mode aspect


Inspect metrics, confusion matrices, and misclassified samples in the generated `outputs/baseline/...` directories.
